In [1]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings  
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
#Since we are using text file here, textloader libararie is used to read the file 
loader=TextLoader(
    file_path="internal_manual.txt")
#we are loading the the data into the lanhchain document using load function what it does is takes the lanhchain object andguives the text in the list form 
data=loader.load()

In [3]:
#since the can be big and llm have limitataions how much they can read we have splite the entire data into chunks and store them 
#so here we are using the recrusive text splitter which helps maiantis the semantic meanings by allowing the over lapping 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
texts = text_splitter.split_documents(data)

In [4]:
#next thiung is we have to create the embeddings for the text for that we need an transfomer model and since we are running an locally everything we are using an HuggingFaceEmbeddings which is freeensures zero API cost and high portability.
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [5]:
#the we need the database to store vector for that we are using an chroma and chromadb chroma takes the texts and emedding model and conevrts them into vectors and stores in the vector databse chromadb 
from langchain_chroma import Chroma
db = Chroma.from_documents(
    documents=texts, 
    embedding=hf,
    persist_directory="chroma_db"
)
 

In [12]:
#ok now we have done the text loading, splitting,and embeddings and piplien to use convert the text into embeddings and vector database, now we need an llm to generate the response of our question 
from langchain_huggingface import HuggingFacePipeline
hf_llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-small",
    task="text2text-generation", # The correct, flexible task
    pipeline_kwargs={"max_new_tokens": 50},  
)

Device set to use cpu


In [7]:
#here we are selecting the search type to be mmr because to avoid the reduduncy maximal marginal redudnace reduce sleceting the same info type documents and makes sures that new document that is selected contains new information 
#Balances relevance and diversity: MMR calculates a score that considers both how similar a document is to the original query and how different it is from documents already chosen. 
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10}, 
)

In [13]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_llm(llm=hf_llm, retriever=retriever)

In [14]:
qa_chain.invoke({"query": "tell me about the warranty duration"})

{'query': 'tell me about the warranty duration',
 'result': 'thirty-six (36) months'}

In [17]:
qa_chain.invoke({"query": "Explain me about the Limited Product Warranty Policy, breifly exaplin "})

{'query': 'Explain me about the Limited Product Warranty Policy, breifly exaplin ',
 'result': 'The warranty specifically excludes any claims arising from user-induced modifications, data'}